# Support and Setup

## Helper methods
TODO - some of these I don't use and should remove.

TODO - there are other helper methods scattered through the code, which would be good to move here.

In [ ]:
import functools

def std_basis(i,dim,power=1):
    # returns (0,...,0,power,0,...,0) in ZZ^dim
    vec = [0]*dim
    vec[i] = power
    return vector(vec)

# list + list is concatenation, here's a method to add lists termwise:
# needs the library functools for reduce.
def add_termwise(*args):
    return list(reduce(lambda x,y: vector(x)+vector(y), args))

# multiply_termwise(v,w) returns (v1*w1,...,vn*wn)
def multiply_termwise(*args):
    return reduce(lambda vec1, vec2: list(map(lambda x,y: x*y, vec1,vec2)), args)
    

def get_normal_ordering_power(term,relations):
    # :ab: = q^{-n}:a: :b:, if ab = q^{2n}ba
    dim = len(vector(ZZ,term))
    scalar_power = 0
    for i in range(dim):
        if not (term[i].is_zero() or vector(term[i+1:]).is_zero()):
            leading_factor = vector([0]*dim)
            leading_factor[i] = term[i]
            trailing_terms = vector([0]*dim)
            trailing_terms[i+1:] = term[i+1:]
        
            scalar_power -= (matrix(leading_factor)*relations*matrix(trailing_terms).transpose())[0]
    return (scalar_power/2)[0]

def add_normal_ordering_scalar(list_of_terms,relations):
    new_list_of_terms = list_of_terms
    for term in new_list_of_terms:
        term[0] += get_normal_ordering_power(term,relations)
    return new_list_of_terms

def dict_monomial_to_list(monomial):
    coeff = LaurentPolynomialRing(ZZ,'qrt4')(list(monomial.values())[0])
    if not coeff.is_monomial():
        raise Exception('I can only deal with coefficients in the form q^a')
    else:
        lattice_coord = list(list(monomial.keys())[0])
        lattice_coord[0] += coeff.degree()/2 # the coeff is in terms of qrt4, qrt4^4 = q
        return lattice_coord

## Lattice Coordinates and Weights.

In [ ]:
import snappy
M = snappy.Triangulation('4_1')
M._two_to_three(0,1)
num_tet = M.num_tetrahedra()

Here's a commented version explaining some about the output of M._to_string().

```

3                                       # <-- number of tetrahedron

# tetrahedron 1
   1    2    1    2                     # <-- which tetrahedron face 0 1 2 3 goes too. 
 1302 3012 0132 0132                    # <-- permutations on the vertices that defines
   0    0    0    0                     #     exactly where the face goes.
  0  0  0  0  0  0  0  0  0  1  0 -1  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 -1  0  1 -1  0  1  0  0  1  0 -1  0  1 -1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0.0 0.0

# tetrahedron 2
   2    0    2    0 
 1302 2031 0132 0132
   0    0    0    0 
  0  0  0  0  1  0 -1  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  2  0 -1 -1 -1  0  0  1  0  1 -1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0.0 0.0

# tetrahedron 3
   0    1    0    1 
 1230 2031 0132 0132
   0    0    0    0 
  0  0  0  0 -1  0  0  1  0  0  0  0  0 -1  1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1 -1  0 -1  0  0  1 -1  0  0  1  1 -2  1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0.0 0.0
```

In [ ]:
# coefficients - helps translate from vectors to elements in the torus.
q = var('q')
qrt2 = var('qrt2')
qrt4 = var('qrt4')
q = qrt4^4
q = qrt2^2
qrt2 = qrt4^2


# TODO - make this ring?
R.<qrt4> =LaurentPolynomialRing(ZZ)
coeffs = matrix(R,[qrt2^2])


### Lattice Coordinates for Short and Long Edges

In [ ]:
# First q. The first lattice is in terms of qrt2 = q^(1/2)
gens_dict = {'qrt2': vector([2] + [0]*(3*18) + [0]*(3*12))}

# Generators for the tetrahedron:
for t in range(M.num_tetrahedra()):
    # short edges
    gens_dict.update(
        {                        # q + before          short                       long + after + threads
            "a{0}01".format(t) : vector([0]*(1+t*18) + [1,0,0,0,0,0,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}02".format(t) : vector([0]*(1+t*18) + [0,1,0,0,0,0,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}03".format(t) : vector([0]*(1+t*18) + [0,0,1,0,0,0,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}10".format(t) : vector([0]*(1+t*18) + [0,0,0,1,0,0,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}13".format(t) : vector([0]*(1+t*18) + [0,0,0,0,1,0,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}12".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,1,0,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}20".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,1,0,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}21".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,0,1,0,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}23".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,0,0,1,0,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}30".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,0,0,0,1,0,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}32".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,0,0,0,0,1,0] + [0]*(6 +18*(2-t) +3*12)),
            "a{0}31".format(t) : vector([0]*(1+t*18) + [0,0,0,0,0,0,0,0,0,0,0,1] + [0]*(6 +18*(2-t) +3*12))
        })
    gens_dict.update(
        {
        "A{0}01".format(t) : vector([0]*(1 +t*18 +12) + [1,0,0,0,0,0] + [0]*(18*(2-t) +3*12)),
        "A{0}02".format(t) : vector([0]*(1 +t*18 +12) + [0,1,0,0,0,0] + [0]*(18*(2-t) +3*12)),
        "A{0}03".format(t) : vector([0]*(1 +t*18 +12) + [0,0,1,0,0,0] + [0]*(18*(2-t) +3*12)),
        "A{0}12".format(t) : vector([0]*(1 +t*18 +12) + [0,0,0,1,0,0] + [0]*(18*(2-t) +3*12)),
        "A{0}13".format(t) : vector([0]*(1 +t*18 +12) + [0,0,0,0,1,0] + [0]*(18*(2-t) +3*12)),
        "A{0}23".format(t) : vector([0]*(1 +t*18 +12) + [0,0,0,0,0,1] + [0]*(18*(2-t) +3*12))
        })
    # I might want to add the swapped indices?
    # gens_dict.update(
     #   {
     #       "A{0}10".format(t) : gens_dict["A{0}01".format(t)],
     #       "A{0}20".format(t) : gens_dict["A{0}02".format(t)],
     #       "A{0}30".format(t) : gens_dict["A{0}03".format(t)],
     #       "A{0}21".format(t) : gens_dict["A{0}12".format(t)],
     #       "A{0}31".format(t) : gens_dict["A{0}13".format(t)],
     #       "A{0}32".format(t) : gens_dict["A{0}23".format(t)]
     #   })
    
# Finally the threads

lex_order_41 = list(gens_dict.keys())
lex_order_41[0] = 'qrt4'
# shorthand:
lat = gens_dict



In [ ]:
# scratch - check vertices
matrix_plot([list(v) for v in gens_dict.values()])

### Vertices

In [ ]:
# Vertices

vertices_dict = {}

# this list isn't used - I just do it by hand.
"""
vertex_labels = []
for t in range(3):
    faces = [0,1,2,3]
    for f in faces:
        punctures = faces.copy()
        punctures.remove(f)
        for p in punctures:
            last_index = punctures.copy()
            last_index.remove(p)
            for i in last_index:
                vertex = "v{0}{1}{2}".format(t,p,i)
                vertex_labels.append(vertex) if vertex_labels.count(vertex) == 0 else True
"""

# make a basis for the vertices. This will be useful for the weights matrix.
for t in range(3):
    vertices_dict.update(
        {                       
            "v{0}01".format(t) : [0]*(12*t) + [1,0,0,0,0,0,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}02".format(t) : [0]*(12*t) + [0,1,0,0,0,0,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}03".format(t) : [0]*(12*t) + [0,0,1,0,0,0,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}10".format(t) : [0]*(12*t) + [0,0,0,1,0,0,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}13".format(t) : [0]*(12*t) + [0,0,0,0,1,0,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}12".format(t) : [0]*(12*t) + [0,0,0,0,0,1,0,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}20".format(t) : [0]*(12*t) + [0,0,0,0,0,0,1,0,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}21".format(t) : [0]*(12*t) + [0,0,0,0,0,0,0,1,0,0,0,0] + [0]*(12*(2-t)),
            "v{0}23".format(t) : [0]*(12*t) + [0,0,0,0,0,0,0,0,1,0,0,0] + [0]*(12*(2-t)),
            "v{0}30".format(t) : [0]*(12*t) + [0,0,0,0,0,0,0,0,0,1,0,0] + [0]*(12*(2-t)),
            "v{0}32".format(t) : [0]*(12*t) + [0,0,0,0,0,0,0,0,0,0,1,0] + [0]*(12*(2-t)),
            "v{0}31".format(t) : [0]*(12*t) + [0,0,0,0,0,0,0,0,0,0,0,1] + [0]*(12*(2-t))
        })
    

    

In [ ]:
# scratch - visualize the vertices basis 
matrix_plot([list(v) for v in vertices_dict.values()])

### Weights for Short and Long Edges

In [ ]:
# Weights
weights_dict = {}
for t in range(M.num_tetrahedra()):
    weights_dict.update({
        'a{0}01'.format(t) : {'v{0}02'.format(t): 1, 'v{0}03'.format(t): -1},
        'a{0}02'.format(t) : {'v{0}03'.format(t): 1, 'v{0}01'.format(t): -1},
        'a{0}03'.format(t) : {'v{0}01'.format(t): 1, 'v{0}02'.format(t): -1},
        'a{0}10'.format(t) : {'v{0}13'.format(t): 1, 'v{0}12'.format(t): -1},
        'a{0}13'.format(t) : {'v{0}12'.format(t): 1, 'v{0}10'.format(t): -1},
        'a{0}12'.format(t) : {'v{0}10'.format(t): 1, 'v{0}13'.format(t): -1},
        'a{0}20'.format(t) : {'v{0}21'.format(t): 1, 'v{0}23'.format(t): -1},
        'a{0}21'.format(t) : {'v{0}23'.format(t): 1, 'v{0}20'.format(t): -1},
        'a{0}23'.format(t) : {'v{0}20'.format(t): 1, 'v{0}21'.format(t): -1},
        'a{0}30'.format(t) : {'v{0}32'.format(t): 1, 'v{0}31'.format(t): -1},
        'a{0}32'.format(t) : {'v{0}31'.format(t): 1, 'v{0}30'.format(t): -1},
        'a{0}31'.format(t) : {'v{0}30'.format(t): 1, 'v{0}32'.format(t): -1}
    })
    
    weights_dict.update({
        'A{0}01'.format(t) : {'v{0}01'.format(t): 1, 'v{0}10'.format(t): 1},
        'A{0}02'.format(t) : {'v{0}02'.format(t): 1, 'v{0}20'.format(t): 1},
        'A{0}03'.format(t) : {'v{0}03'.format(t): 1, 'v{0}30'.format(t): 1},
        'A{0}12'.format(t) : {'v{0}12'.format(t): 1, 'v{0}21'.format(t): 1},
        'A{0}13'.format(t) : {'v{0}13'.format(t): 1, 'v{0}31'.format(t): 1},
        'A{0}23'.format(t) : {'v{0}23'.format(t): 1, 'v{0}32'.format(t): 1},
    })
    


### Threads and Gluing Data
Including weights and lattice coordinates for threads

In [ ]:
def get_short_edges_in_face(tet,face):
    """
    Returns a list of the short edges contained in a specified face.

    Parameters:
    tet (Integer) - index of the tetrahedron containing the face.
    face (Integer) - index of the face.

    Returns:
    list of str - list of short edges contained in the face.
    """
    edge_indices = [0,1,2,3]
    edge_indices.remove(face)
    return ["a{0}{1}{2}".format(tet,i,face) for i in edge_indices]

def get_long_edges_in_face(tet,face):
    """
    Returns a list of the long edges contained in a specified face.

    Parameters:
    tet (Integer) - index of the tetrahedron containing the face.
    face (Integer) - index of the face.

    Returns:
    list of str - list of long edges contained in the face.
    """
    indices = [0,1,2,3]
    indices.remove(face)
    return ["A{0}{1}{2}".format(tet,indices[i1],indices[i2]) for i1 in range(3) for i2 in range(i1+1,3)]   
        
        
    return ["A{0}{1}{2}".format(tet,i,face) for i in edge_indices]

def get_long_edges_in_tet(tet):
    """
    Returns a list of the long edges contained in a specified face.

    Parameters:
    tet (Integer) - index of the tetrahedron.

    Returns:
    list of str - list of long edges contained in the tet.
    """
    return [name.format(tet) for name in ['A{0}01','A{0}02', 'A{0}03','A{0}12','A{0}13','A{0}23']]
    
    
def get_local_thread_weights(tet,face):
    """
    Finds the weights for the threads that would glue this face to another.

    Parameters:
    tet (Integer) - index of the tetrahedron containing the face
    face (Integer) - index of the face inside the tetrehedron.

    Returns:
    dict {str : Integer} - gives the weights of the threads for each vertex in the face."""
    short_edges = get_short_edges_in_face(tet,face)
    local_thread_weights = {}
    for short_edge in short_edges:
        local_thread_weights.update({
            vertex: -weight for vertex,weight in weights_dict[short_edge].items()
        })

    return local_thread_weights


In [ ]:
get_long_edges_in_tet(0)

In [ ]:
# turn the gluing data into a dictionary.

def get_gluing_dict(M):
    """
    Reformats the gluing data from a snappy triangulation with n tetrahedron.

    Parameters:
    M (snappy.Triangulation)

    Returns
    dict - each item is a permutation:
        'r{i}' specifies which tetrahedrons the faces of the i-th tet ends up in.
        's{i}{j}' specifies the gluing map for the j-th face of the i-th tet.
    """
    gluing_data = M._get_tetrahedra_gluing_data()
    gluing_dict = {}
    for t in range(M.num_tetrahedra()):
        gluing_dict.update({"r{0}".format(t) : gluing_data[t][0] })
        for f in range(4):
            gluing_dict.update({"s{0}{1}".format(t,f) : gluing_data[t][1][f]})
            
    return gluing_dict



# This list isn't used - it's replaced by get_thread_weights_dict.
"""
thread_endpoints = []
for t in range(3):
    for f in range(4):
        second_index = [0,1,2,3]
        second_index.remove(f)
        face_perm = gluing_dict["s{0}{1}".format(t,f)]
        for i2 in second_index:
            third_index = second_index.copy()
            third_index.remove(i2)
            for i3 in third_index:
                local_vertex = "v{0}{1}{2}".format(t,i2,i3)
                distant_vertex = "v{0}{1}{2}".format(
                    gluing_dict["r{0}".format(t)][f],
                    face_perm[i2],
                    face_perm[i3]
                )
                thread_endpoints.append({local_vertex, distant_vertex}) if thread_endpoints.count({local_vertex,distant_vertex}) == 0 else True
""";

In [ ]:
# This could be more beautiful.
def get_thread_weights_dict(M):
    """
    Finds the weights_dict for the threads, based on a snappy Triangulation.
    """
    gluing_dict = get_gluing_dict(M)
    thread_weights_dict = {}
    for t in range(M.num_tetrahedra()):
        for f in range(4):
            face_perm = gluing_dict["s{0}{1}".format(t,f)]
            local_thread_weights = get_local_thread_weights(t,f)
            # add threads based on where they start.
            start_here = list(filter(lambda k : local_thread_weights[k] == 1, local_thread_weights.keys()))
            for local_vertex in start_here:
                distant_vertex = "v{0}{1}{2}".format(
                    gluing_dict["r{0}".format(t)][f],
                    face_perm[Integer(local_vertex[-2])],
                    face_perm[Integer(local_vertex[-1])]
                )
                thread_weights_dict.update({
                    "x{0}_{1}".format(local_vertex[1:], distant_vertex[1:]) : {local_vertex: 1, distant_vertex: -1}   
                })

    return thread_weights_dict


# Add thread weights to the big weights dictionary.
weights_dict.update(get_thread_weights_dict(M))

In [ ]:
weights_dict;

In [ ]:
# SCRATCH - figuring out the thread weights
short_edges = get_short_edges_in_face(0,3)

local_thread_weights = {}
for short_edge in short_edges:
    local_thread_weights.update({
        vertex: -weight for vertex,weight in weights_dict[short_edge].items()
    })
   # for vertex,weight in weights_dict[short_edge].items():
    #    local_thread_weights[vertex] = -weight
  
#print(short_edges)
#print(local_thread_weights)
ltw = get_local_thread_weights(0,3)
start_here = list(filter(lambda k : ltw[k] == -1, ltw.keys()))

start_here
#print(tmp)

#### Lattice Coordinates for Threads

In [ ]:
def add_thread_lattice_coordinates(M,gens_dict,weights_dict):
    """ Assumes the the thread weights have already been added. """
    num_tet = M.num_tetrahedra()
    thread_names = list(filter(lambda k : k[0] == 'x', weights_dict.keys()))
    for i in range(len(thread_names)):
        gens_dict.update({
            thread_names[i] : vector([0]*(1+18*num_tet+i) + [1] + [0]*(12*num_tet-i-1))
        })

add_thread_lattice_coordinates(M,gens_dict,weights_dict)

## Relations

### Relations for Short and Long Edges.

In [ ]:
# first do short edges around a puncture
omega_short_one_puncture = matrix([
    [ 0, 1,-1],
    [-1, 0, 1],
    [ 1,-1, 0]
])
omega_short = matrix.block_diagonal([omega_short_one_puncture]*4)

# columns are At01, At02, At03, At12, At13, At23
omega_short_long = matrix([
    [0,1,1,0,0,0],
    [1,0,1,0,0,0],
    [1,1,0,0,0,0],
    #
    [0,0,0,1,1,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,0],
    #
    [0,0,0,1,0,1],
    [0,1,0,0,0,1],
    [0,1,0,1,0,0],
    #
    [0,0,0,0,1,1],
    [0,0,1,0,1,0],
    [0,0,1,0,0,1]
])



# columns are short then long
omega_one_tet = block_matrix([
    [omega_short,omega_short_long],
    [-omega_short_long.transpose(),0]
])

In [ ]:
# SCRATCH
omega_one_tet

### Thread Relations

In [ ]:
def get_edges_adjacent_to_vertex(vertex,weights_dict):
    """
    Returns a list of generators with non-zero weight at a given vertex.
    
    Parameters:
    vertex (str) - the name of a vertex.
    weights_dict (dict) - the weights dictionary.
    
    Returns:
    list of strings - the names of all edges with non-zero weight at this vertex.
    """
    adjacent_edges = []
    for edge, weights in weights_dict.items():
        if vertex in weights and weights[vertex] != 0:
            adjacent_edges.append(edge)
            
    return adjacent_edges

def get_thread_relations(gens_dict,weights_dict):
    """
    Builds the part of the relations matrix concerning threads.
    """
    thread_names = list(filter(lambda k : k[0] == 'x', gens_dict.keys()))
    thread_relations = []
    for thread in thread_names:
        this_threads_relations = []
        for vertex,weight in weights_dict[thread].items():
            tmp_adjacent_edges = get_edges_adjacent_to_vertex(vertex,weights_dict)
            tmp_adjacent_edges.remove(thread)
            for edge in tmp_adjacent_edges:
                # commutation relation is the same for all non-thread generators at this vertex.
                if edge[0] == 'x':
                    this_threads_relations.append(weight*gens_dict[edge])
                else:
                    this_threads_relations.append(-weight*gens_dict[edge])
        thread_relations.append(sum(this_threads_relations))
    
    return matrix(thread_relations)


thread_relations = get_thread_relations(gens_dict,weights_dict)
# split the thread relations into two parts, to help construct the full relations matrix.
omega_thread_non_thread = thread_relations[:,1:18*3+1]
omega_thread_thread = thread_relations[:,18*3+1:]


In [ ]:
#checks

print(thread_relations[:,18*3+1:].is_skew_symmetric())
omega_thread_non_thread


### Full Relations
TODO? Maybe wrap this in a function so it's easier to change triangulations.

In [ ]:
# Get the relations matrix, then its kernel
omega_41_with_q = block_diagonal_matrix(
    matrix([0]),
    block_matrix([
        [block_diagonal_matrix(*[omega_one_tet]*num_tet),-omega_thread_non_thread.transpose()],
        [omega_thread_non_thread,omega_thread_thread]
    ])
)

kernel_41 = omega_41_with_q.kernel().basis()

Here are some sanity checks for the relations matrix:

In [ ]:
# checking that the keys are in the same order.
print("weights_dict and gens_dict have the same order:", list(weights_dict.keys())==list(gens_dict.keys())[1:])

print("omega_41 is skew symmetric:", omega_41_with_q.is_skew_symmetric())

print("Rank of the center:",omega_41_with_q.right_kernel_matrix().rank())

matrix_plot(omega_41_with_q)

## The Invariant Sublattice
We also make the weight _matrix_ in this section.

### Weight Matrix

In [ ]:
# start off with qrt2 - this is weight zero.
weights_list = [[0]*(len(vertices_dict.keys()))] 
for weights in weights_dict.values():
    weights_list.append(sum([weight*vector(vertices_dict[vertex]) for vertex,weight in weights.items()]))
 
weight_matrix = Matrix(weights_list)

invariant_sublattice = weight_matrix.left_kernel()

In [ ]:
# Some checks.

print("Basis for the T-invariant sub-lattice:")
show(matrix_plot(invariant_sublattice.basis_matrix()))
print("The Weight Matrix:")
show(matrix_plot(weight_matrix))

# Quotients

Any quotient of the form $M^{x} = 1$ in the quantum torus can be expressed in terms of the lattice as $x = 0$. We want to quotient by
1. Monodromy at each of the 8 punctures
1. The gluing constraints, 6 from each face so 24 total.
1. Monodromy around the long edge handles - looks like there are 2 from each edge so 4 total.
1. The great circle relations, 1 from each tetrahedron so 2 total.

The first three types of relations are monomial. I expect to get a lattice of rank 3 once those are imposed. The last relation will have to be done in terms of the quantum torus, not the lattice.


## Constraints Lists

### T-Monodromy Around Punctures
TODO - add correct power of q. 

In [ ]:
def get_T_monodromy_list(M,gens_dict,num_tet):
    num_tet = M.num_tetrahedra()
    # the short edges are listed in gens_dict in the right order.
    short_edge_names = list(filter(lambda name: name[0]=='a', gens_dict.keys()))
    T_monodromy_list = []
    for p in range(num_tet*4):
        T_monodromy_list.append(short_edge_names[3*p:3*p+3])
    
    return T_monodromy_list

T_monodromy_variable_names_list = get_T_monodromy_list(M,gens_dict,num_tet)
T_monodromy_lattice_coordinate_list = [
    sum([
        gens_dict[name] for name in monodromy
    ]) for monodromy in T_monodromy_variable_names_list
]

In [ ]:
# Checks.
for coord in T_monodromy_lattice_coordinate_list:
    if not (matrix(coord)*weight_matrix).is_zero():
        print("Not Invariant:",coord)
matrix_plot(T_monodromy_lattice_coordinate_list)

### Gluing Relations
TODO - add correct power of q.

In [ ]:
def get_long_edge_gluing_relations_list(M,gens_dict):
    long_edge_gluing_relations_list = []
    for tet in range(M.num_tetrahedra()):
        for long_edge in get_long_edges_in_tet(tet):
            thread1 = [thread_name for thread_name in gens_dict.keys() 
                       if 'x'+long_edge[1:] in thread_name
                      ][0]
            distant_vertex_index = thread1.split('_')[1]
            # the long edge At01 connects vt01 and vt10
            # and is adjacent to threads xt01_??? and xt10_???
            distant_long_edge = 'A' + distant_vertex_index[:-2]+ ''.join(sorted(distant_vertex_index[-2:]))
            # reverse the last two digits to get the other end of the long edge
            distant_thread_index = distant_vertex_index[:-2]+ ''.join(list(reversed(distant_vertex_index[-2:])))
            thread2 = [thread_name for thread_name in gens_dict.keys() 
                       if thread_name.startswith('x'+distant_thread_index)
                      ][0]

            long_edge_gluing_relations_list.append([long_edge,thread1,distant_long_edge,thread2])
    return long_edge_gluing_relations_list

long_edge_gluing_relations_list = get_long_edge_gluing_relations_list(M,gens_dict)

In [ ]:
# Check that all relations are T-invariant, and incidentally that we've listed actual threads.

for constraint in long_edge_gluing_relations_list:
    lat_coord = -gens_dict[constraint[0]]+gens_dict[constraint[1]]+gens_dict[constraint[2]]-gens_dict[constraint[3]]
    if not (lat_coord*weight_matrix).is_zero():
        print("Not Invariant!", lat_coord*weight_matrix)

In [ ]:
# SCRATCH - I'm trying to get the equivalence classes of long edges. This is currently broken

for i in range(len(long_edge_gluing_relations_list)):
    equivalence_class = []
    for j in range(i,len(long_edge_gluing_relations_list)):
        intersection = set(equivalence_class) & set(long_edge_gluing_relations_list[j])
        union = set(long_edge_gluing_relations_list[i])|set(long_edge_gluing_relations_list[j])
        if intersection != 0:
            equivalence_class += list(union)
    print(equivalence_class,'\n')

In [ ]:
def get_short_edge_gluing_relations_list(M,weights_dict):
    """
    Constructs a the list of constraints inclured by gluing together short edges.
    
    Parameters:
    M snappy.Triangulation - the triangulated knot complement.
    weights_dict - the weights of edges. Used to find where edges start/end.
    
    Returns:
    list of lists of strings - elements are lists of generator names. Should be in the right order. 
    """
    gluing_data = get_gluing_dict(M)
    short_edge_gluing_relations_list = []
    
    for tet in range(num_tet):
        for face in range(4):
            new_tet = gluing_data['r{0}'.format(tet)][face]
            if new_tet <= tet: # avoid double-lisitng relations. will still double-list self-foldings.
                gluing_perm = gluing_data['s{0}{1}'.format(tet,face)]
                new_face = gluing_perm[face]
                for short_edge in get_short_edges_in_face(tet,face):
                    distant_short_edge = 'a{0}{1}{2}'.format(new_tet,gluing_perm[Integer(short_edge[-2])],new_face)

                    tmp_local_weights = weights_dict[short_edge]
                    local_starting_vertex = [vertex[1:] for vertex,weight in tmp_local_weights.items() if weight == -1][0]  
                    local_ending_vertex = [vertex[1:] for vertex,weight in tmp_local_weights.items() if weight == 1][0]  

                    tmp_distant_weights = weights_dict[distant_short_edge]
                    distant_starting_vertex = [vertex[1:] for vertex,weight in tmp_distant_weights.items() if weight == -1][0]  
                    distant_ending_vertex = [vertex[1:] for vertex,weight in tmp_distant_weights.items() if weight == 1][0]  

                    short_edge_gluing_relations_list.append([
                        short_edge,'x{0}_{1}'.format(local_starting_vertex,distant_ending_vertex),
                        distant_short_edge,
                        'x{0}_{1}'.format(distant_starting_vertex,local_ending_vertex)
                    ])
    return short_edge_gluing_relations_list

short_edge_gluing_relations_list = get_short_edge_gluing_relations_list(M,weights_dict)

In [ ]:
# Check that all relations are T-invariant, and incidentally that we've listed actual threads.
for constraint in short_edge_gluing_relations_list:
    if not (sum([gens_dict[edge] for edge in constraint])*weight_matrix).is_zero():
        print("Not Invariant!", constraint)


## Monodromy and Gluing Quotients - NOT UPDATED


In [ ]:
%run ../quantum_tori.ipynb

In [ ]:
# We'll want to check what's in the kernel:
#kernel_space_41 = invariant_sublattice_41.subspace(kernel_41)

# Make the gluing constraints in a form where the power of q is tracked properly:
qrt2 = qrt4^2

list_of_gluing_constraints_41 = [ # first short edges. The loop is q^(-1) x2 a11 x1 b43 and set equal to q^(-1)
    # the order depends on the orientation around the loop.
    dict_monomial_to_list(constraint) for constraint in [
        scale_polynomial(multiply_lattice_monomials(new_gens(thread1),new_gens(edgeA),new_gens(thread2),new_gens(edgeB),relations=omega_41_with_q),q^(0))
        for thread1,edgeA,thread2,edgeB in [
            ('x2','al03','x1','be02'),
            ('x4','al02','x3','be03'),
            ('x6','al01','x5','be01'),
            ('y1','al13','y2','be32'),
            ('y3','al12','y4','be30'),
            ('y5','al10','y6','be31'),
            ('z2','al23','z1','be23'),
            ('z4','al21','z3','be20'),
            ('z6','al20','z5','be21'),
            ('w1','al32','w2','be13'),
            ('w3','al31','w4','be10'),
            ('w5','al30','w6','be12')
        ]
    ]
] + [ # then long edges. These get set to 1. It's :x2 A4:(:x3 B6:)^{-1} = 1.
    dict_monomial_to_list(constraint) for constraint in [ 
        multiply_polynomials({ tuple(new_gens(thread1)+new_gens(edgeA)):1 },{tuple(-new_gens(thread2)-new_gens(edgeB)):1},relations=omega_41_with_q)
        for thread1,edgeA,thread2,edgeB in [
            ('x2','B23','x3','A23'),
            ('x4','B13','x5','A12'),
            ('x6','B12','x1','A13'),
            ('y2','A23','y3','B02'),
            ('y4','A02','y5','B01'),
            ('y6','A03','y1','B12'),
            ('z2','B03','z3','A13'),
            ('z4','B01','z5','A01'),
            ('z6','B13','z1','A03'),
            ('w2','A12','w3','B03'),
            ('w4','A01','w5','B02'),
            ('w6','A02','w1','B23')
        ]
    ]
]

# check that these are T^{24}-invariant and central:
print("Checking Gluing Constraints:")
for constraint in list_of_gluing_constraints_41:
    v = weights_41*vector(constraint)
    if not v.is_zero():  print("Not Invariant!:", new_coeffs*matrix(Rnew,constraint).transpose())
#    if not (invariant_sublattice_41.subspace([constraint])).is_subspace(kernel_space_41): print("Not in kernel:", constraint)


# Monodromy: The loop around the puncture is q^(-1/2) a1p a2p a3p. If it's set to q^(-1/2), then I need to quotient by ap1 ap2 ap3 = q^(0)
    
list_of_monodromy_constraints_41 = [ dict_monomial_to_list(constraint) for constraint in [ 
    scale_polynomial(multiply_lattice_monomials(new_gens(edge1),new_gens(edge2),new_gens(edge3),relations=omega_41_with_q),q^(0))
    for edge1,edge2,edge3 in [
            ('al10','al20','al30'),
            ('al01','al21','al31'),
            ('al02','al12','al32'),
            ('al03','al13','al23'),
            ('be10','be20','be30'),
            ('be01','be21','be31'),
            ('be02','be12','be32'),
            ('be03','be13','be23'),
        ]
    ]
]

# check that these are T^{24}-invariant
print("Checking Monodromy Constraints:")
for constraint in list_of_monodromy_constraints_41:
    v = weights_41*vector(constraint)
    if not v.is_zero():  print("Not Invariant!:", new_coeffs*matrix(Rnew,constraint).transpose())
    # to check if they're central. I don't think they should be.
    #if not (invariant_sublattice_41.subspace([constraint])).is_subspace(kernel_space_41): print("Not in kernel:", constraint)
    
# Take the quotient!
#quotient_41 = invariant_sublattice_41.quotient(list_of_monodromy_constraints_41+list_of_gluing_constraints_41)

#print("Ambient space dimensions:",invariant_sublattice_41.dimension())
#print("Constraint space dimensions:",invariant_sublattice_41.subspace(list_of_monodromy_constraints_41+list_of_gluing_constraints_41).dimension())
#print("Quotient ngens:",quotient_41.ngens())

## Full Quotient!  - NOT UPDATED
We expect this to have three generators.

In [ ]:
%run ../quantum_tori.ipynb
list_of_thread_monodromy_constraints_41 = [ # the loop is q^(-1/2)(threads in ccw order) -> q^(-1/2)
    dict_monomial_to_list(constraint) for constraint in [
        scale_polynomial(multiply_lattice_monomials(*[lat[e] for e in edges],relations=omega_41_with_q),1)
        for edges in [
            ('x1', 'x4', 'y1', 'z1', 'z2', 'w2'),
            ('x2', 'y2', 'y5', 'z5', 'w5', 'w6'),
            ('x3', 'y3', 'y4', 'z4', 'w1', 'w4'),
            ('x5', 'x6', 'y6', 'z3', 'z6', 'w3')
        ]
    ]
]

# check that these are T^{24}-invariant 
for constraint in list_of_thread_monodromy_constraints_41:
    v = weights_41*vector(constraint)
    if not v.is_zero():  print("Not Invariant!:", constraint)
        
full_quotient_41 = invariant_sublattice_41.quotient(list_of_gluing_constraints_41+list_of_monodromy_constraints_41+list_of_thread_monodromy_constraints_41)
print("Full (threads+monodromy+gluing) quotient ngens: ", full_quotient_41.ngens())

pi = full_quotient_41.coerce_map_from(full_quotient_41.V())
for g in full_quotient_41.gens():
    print(new_coeffs*vector(Rnew,list(g.lift())))

    

In [ ]:
# lets print all the constraints
# All of the T-region monodromy constraints should be set to qdim(1). 
print("thread monodromy.")
for constraint in list_of_thread_monodromy_constraints_41:
    print(new_coeffs*vector(Rnew,constraint))
    
print("monodromy.")
for constraint in list_of_monodromy_constraints_41:
    print(new_coeffs*vector(Rnew,constraint))
    
print("gluing.")
for constraint in list_of_gluing_constraints_41:
    print(new_coeffs*vector(Rnew,constraint))

## The great circles  - NOT UPDATED

These are *not* monomials, so I can't take the quotient using the lattice. Here I translate each term into the lattice though, to see if they'll simplify.

In [ ]:
%run ../quantum_tori.ipynb

In [ ]:
def list_to_module_element(the_list,the_module):
    """
    Turns a list into a module element.
    Plays the role that the_module.monomial() would if it were implemented.
    
    Parameters:
    the_list (List) - the lattice coordinate of a module element.
    the_module (fgp_module) - the ambient module
    
    Returns:
    (the_module.element_class) - the module element you'd *expect* from typing the_module(the_list)
    """
    return the_module.sum([
        term[0]*term[1] for term in zip(the_list,the_module.gens())
    ])

In [ ]:
### Generators for the glued surface!

# we want to generate the lattice and have determinant +1 for the basis matrix
extra_gen = list_to_module_element([0,-1,0],full_quotient_41).lift()
# some sanity checks on the extra generator:
if not (weights_41*vector(extra_gen)).is_zero():
    print("weight of extra generator:",weights_41*extra_gen)

print("extra_gen in quotient:",pi(extra_gen))

m_gen = vector(dict_monomial_to_list(
    multiply_lattice_monomials(
        *[new_gens(edge) for edge in ['x1','al03','al13','y1']],
        relations=omega_41_with_q
    )))

if not (weights_41*vector(m_gen)).is_zero():
    print("weight of m:",weights_41*m_gen)
    
print("m_gen in quotient:",pi(m_gen))

l_gen = dict_monomial_to_list(scale_polynomial(
    multiply_lattice_monomials(
        *[new_gens(edge) for edge in ['z2','al23','al13','y2','w5','z5','be21','be01','x6','z3','be20','be30','y3','x3','be03','be23']],
        relations=omega_41_with_q
    ),q^(-1/2)))
if not (weights_41*vector(l_gen)).is_zero():
    print("weight of l:",weights_41*vector(l_gen))
    
print("l_gen in quotient:",pi(l_gen))


favorite_basis_matrix = matrix([
    pi(m_gen),
    pi(l_gen),
    pi(extra_gen)
]).transpose()

if favorite_basis_matrix.determinant() != 1:
    print("WARNING: favorite_basis_matrix has determinant:",favorite_basis_matrix.determinant())
    
    
print("Commutation Relations in the quotient. These are in terms of q^(1/2).") # this is assuming that the quotient preserves the commutation relations.
omega_quotient = matrix([m_gen,l_gen,extra_gen])*omega_41_with_q*matrix([m_gen,l_gen,extra_gen]).transpose()
print(omega_quotient)

tmp_omega = (omega_41_with_q*matrix([m_gen,l_gen]).transpose())
#TODO - is there something that commutes with the m and l that I could use as the third generator?

In [ ]:
m_gen

In [ ]:
scafolding_edges_A = -1*(new_gens('A03')+new_gens('A02')+new_gens('A12')+new_gens('A13'))
scafolding_edges_B = -1*(new_gens('B03')+new_gens('B02')+new_gens('B12')+new_gens('B13'))

# Define the great circles as dictionaries. These should be set to q + q^(-1)?

temp_list_A = [
    scale_polynomial( multiply_lattice_monomials(
        *[scafolding_edges_A,*edges],
        relations=omega_41_with_q
    ), q^q_power) for edges,q_power in [
                 # 0010
                 ([lat['A1'],-lat['a33'],lat['A1'],-lat['a42'],-lat['a22'],lat['A3'],lat['A3'],-lat['a13']],1),
                 # 0110
                 ([lat['a23'],lat['A6'],-lat['a41'],lat['A1'],-lat['a42'],lat['a31'],lat['A5'],-lat['a12'],lat['A3'],-lat['a13']],0),
                 #0011
                 ([lat['A1'],-lat['a33'],  lat['a11'],lat['A5'],-lat['a32'],  -lat['a22'],lat['A3'],  lat['a43'],lat['A6'],-lat['a21']],0),
                 # 0111
                 ([lat['a23'],lat['A6'],-lat['a41'],  lat['a11'],lat['A5'],-lat['a32'], lat['a31'],lat['A5'],-lat['a12'],  lat['a43'],lat['A6'],-lat['a21']],-1),
                 # 1101
                 ([lat['a13'],lat['A4'],  lat['a42'],lat['A4'],  lat['a22'],lat['A2'],  lat['a33'],lat['A2']],-1),
                 # 0101
                 ([lat['a13'],lat['A4'],  lat['a11'],lat['A5'],-lat['a32'],  lat['a22'],lat['A2'],  lat['a23'],lat['A6'],-lat['a41']],0),
                 # 1111
                 ([lat['a43'],lat['A6'],-lat['a21'],  lat['a42'],lat['A4'],  lat['a31'],lat['A5'],-lat['a12'],  lat['a33'],lat['A2']],-2)
] ]



great_circle_A = add_polynomials(*temp_list_A)
print("Great Circle:")
pretty_print(dict_to_free_algebra_element(great_circle_A, lex_order=lex_order_41,free_algebra=R))
    
great_circle_quot_A = add_polynomials(
    *[{tuple(favorite_basis_matrix*pi(k).vector()):v} for k,v in great_circle_A.items()]
)

great_circle_B = {}

temp_list_B = [
    scale_polynomial( multiply_lattice_monomials(
        *[scafolding_edges_B,*edges],
        relations=omega_41_with_q
    ), q^q_power) for edges,q_power in [
                 # 0010
                 ([lat['B1'],-lat['b33'],lat['B1'],-lat['b42'],-lat['b22'],lat['B3'],lat['B3'],-lat['b13']],1),
                 # 0110
                 ([lat['b23'],lat['B6'],-lat['b41'],lat['B1'],-lat['b42'],lat['b31'],lat['B5'],-lat['b12'],lat['B3'],-lat['b13']],0),
                 #0011
                 ([lat['B1'],-lat['b33'],  lat['b11'],lat['B5'],-lat['b32'],  -lat['b22'],lat['B3'],  lat['b43'],lat['B6'],-lat['b21']],0),
                 # 0111
                 ([lat['b23'],lat['B6'],-lat['b41'],  lat['b11'],lat['B5'],-lat['b32'], lat['b31'],lat['B5'],-lat['b12'],  lat['b43'],lat['B6'],-lat['b21']],-1),
                 # 1101
                 ([lat['b13'],lat['B4'],  lat['b42'],lat['B4'],  lat['b22'],lat['B2'],  lat['b33'],lat['B2']],-1),
                 # 0101
                 ([lat['b13'],lat['B4'],  lat['b11'],lat['B5'],-lat['b32'],  lat['b22'],lat['B2'],  lat['b23'],lat['B6'],-lat['b41']],0),
                 # 1111
                 ([lat['b43'],lat['B6'],-lat['b21'],  lat['b42'],lat['B4'],  lat['b31'],lat['B5'],-lat['b12'],  lat['b33'],lat['B2']],-2)
    ] ]

great_circle_B = add_polynomials(*temp_list_B)

great_circle_quot_B = add_polynomials(
    *[{tuple(favorite_basis_matrix*pi(k).vector()):v} for k,v in great_circle_B.items()]
)



In [ ]:
Qq.<qrt4> = FractionField(QQ['qrt4'])
var('q')
q = qrt4^4

quotient_free_algebra = FreeAlgebra(Qq,['m','l','g'])
quotient_poly_algebra = LaurentPolynomialRing(Qq,['m','l','g'])
classical_poly_algebra = LaurentPolynomialRing(QQ,['m','l','g'])
quotient_lex_order = ['m','l','g']

# Add -q^(1/2) - q^(-1/2) and turn into an element of the laurent polynomial ring.
print("Circle A:")
relation_A = quotient_poly_algebra.sum([
    quotient_poly_algebra(v)*quotient_poly_algebra.monomial(*k) for k,v in great_circle_quot_A.items()
]+[-qrt4^2 + -qrt4^(-2)])
pretty_print(relation_A)

print("Circle B:")
relation_B = quotient_poly_algebra.sum([
    quotient_poly_algebra(v)*quotient_poly_algebra.monomial(*k) for k,v in great_circle_quot_B.items()
]+[-qrt4^2 - qrt4^(-2)])
pretty_print(relation_B)

# ideal:

relations_ideal = quotient_poly_algebra.ideal([relation_A,relation_B])

In [ ]:
relation_A, relation_B


## Set q->1 to get the classical expression.  - NOT UPDATED

In [ ]:
# I want to compare to the q=1 case.
f = Qq.hom(1,QQ) # map sending qrt4 -> 1.

classical_relation_A = relation_A.map_coefficients(f)
    
classical_relation_B = relation_B.map_coefficients(f)

print("Classical relations:")
pretty_print(classical_relation_A)
pretty_print(classical_relation_B)

## Searching for the A-polynomial - scratch  - NOT UPDATED

In [ ]:
# short names for convenience
r1 = classical_relation_A
r2 = classical_relation_B

pretty_print(r1.factor())
pretty_print(r2.factor())

classical_poly_algebra.ideal([r1,r2]).groebner_basis()

In [ ]:
# Check that all the terms in the great circles are invariant.
for k in great_circle_A.keys():
    if not (weights_41*matrix(k).transpose()).is_zero():
        print("Not invariant:", k)
        
for k in great_circle_B.keys():
    if not (weights_41*matrix(k).transpose()).is_zero():
        print("Not invariant:", k)

In [ ]:
#import sage.rings.polynomial.laurent_polynomial_ideal
#TODO: Figure out how to make ideals in a laurent polynomial ring!
Rquot.<M,L,G> = PolynomialRing(QQ,3) # Order of variables here informs the Groebner basis.
M,L,G = var('M','L','G')

In [ ]:
# I'm adding 2 since the great circles should be set to q^(1/2)+q^(-1/2)
# and we set q^(1/2) to -1. 
#r1 = 2 + q^(-1/2)*M*G + (1+q^(1/2))*M^(-1)*L^(-1) + q*M^(-3)*L^(-2)*G^(-1) + q^2*M^(-1)*G^(-1) + (q^(5/2)+q^(1/2))*M^(-2)*L^(-1)*G^(-1)     
#r1rescaled = (r1.subs(qrt4=i)*G).collect(G)
#r2 = 2 + q^(-1/2)*M^(-3)*G + (1+q^(1/2))*L*G + q*M^(3)*L^(2)*G^(1) + q^2*M^(3)*G^(-1) + (q^(5/2)+q^(1/2))*M^(3)*L^(1)    
#r2rescaled = (r2.subs(qrt4=i)*G).collect(G)

r1 = q^(-1/2) + (1+q^(1/2))*L*G + q*L^2*G^2 + q^2*M^2*L^4*G^2 + (q^(5/2)+q^(1/2))*M*L^3*G^2 - (q^(1/2)+q^(-1/2))*M*L^2*G
r2 = q^(-1/2) + (q+q^(3/2))*M^3*L + q^3*M^6*L^2 + q^2*M^10*L^4*G^2 + (q^(7/2)+q^(3/2))*M^8*L^3*G - (q^(1/2)+q^(-1/2))*M^5*L^2*G

r1_comm = r1.subs(qrt4=i).collect(G)
r2_comm = r2.subs(qrt4=i).collect(G)

pretty_print(r1_comm)
pretty_print(r2_comm)


#r1 = -M^4*L^2*G^2 + 0 + 1 + M^2*L^2 -2*M*L
#r2 = -G^2 + M^6*L^2*G^2 + M^6 -2*M^6*L*G

a2 = r1_comm.coefficient(G,2)
a1 = r1_comm.coefficient(G,1)
a0 = r1_comm.coefficient(G,0)

b2 = r2_comm.coefficient(G,2)
b1 = r2_comm.coefficient(G,1)
b0 = r2_comm.coefficient(G,0)

#dict_to_free_algebra_element(great_circle_quot_A,lex_order=['M','L','G'],free_algebra=Rquot)
#dict_to_free_algebra_element(great_circle_quot_B,lex_order=['M','L','G'],free_algebra=Rquot)
#Iquot.groebner_basis()
Apoly = L^2*M^4 + L*(-M^8+M^6+2*M^4+M^2-1)+M^4
#Aideal = Rquot.ideal(L^2*M^4 + L*(-M^8+M^6+2*M^4+M^2-1)+M^4)
pretty_print(Apoly)

In [ ]:
r1tilde = a2*(b2*a0*b1^(-1)*a2^(-1) - b0*b1^(-1))^2 + a0

pretty_print(r1tilde.collect(M))
pretty_print(r1tilde.collect(L).factor())

In [ ]:
# Can we quotient first?
quotient_first_41 = (ZZ^(60)).quotient(gluing_constraints_41+monodromy_constraints_41)

# weights in the quotient: project each row then make them into a new matrix.
TMP_weights_in_quotient_41 = []
for r in weights_41.rows():
    TMP_weights_in_quotient_41.append(list(quotient_first_41.coordinate_vector(r)))
    
weights_in_quotient_41 = matrix(TMP_weights_in_quotient_41)
print("Rank of the weight function in the quotient:",weights_in_quotient_41.rank())

# now take the invariants. We guess that this is rank 3.
#TODO: Figure out why this has rank 5. Also, figure out what rank the quotient should be.
invariants_in_quotient_41 = weights_in_quotient_41.right_kernel(basis='LLL')

In [ ]:
# Looking at redundant constraints.
print("The",len(monodromy_constraints_41),"monodromy constraints span a sublattice of rank",(ZZ^(60)).span(monodromy_constraints_41).dimension())
print("The",len(gluing_constraints_41),"gluing constraints span a sublattice of rank",(ZZ^(60)).span(gluing_constraints_41).dimension())
print("Together, they span a space of dimension",(ZZ^(60)).span(gluing_constraints_41+monodromy_constraints_41).dimension())
print("ngens: Over QQ, for the quotient with all T-gates open:",(QQ^(60)).quotient(gluing_constraints_41+monodromy_constraints_41).ngens())
print("ngens: Over ZZ, for the quotient with all T-gates open:",(ZZ^(60)).quotient(gluing_constraints_41+monodromy_constraints_41).ngens())

(ZZ^(60)).quotient(gluing_constraints_41+monodromy_constraints_41)

In [ ]:
# name the weight after the thread with weight 1 there. 
gate_x1_weight = matrix(add_termwise(short_edge(1,1,-1),short_edge(1,2,1),long_edge(5)) + [0]*18 + list(x1vec-z2vec))

gate_x2_weight = matrix([0]*18 + add_termwise(short_edge(4,3,-1),short_edge(4,1,1),long_edge(6))+list(x2vec-z5vec))


The moment map will commute with any weight-zero vector, so we can start by restricting to the centralizer.

In [ ]:
# TODO - this might be cleaner if we start with an identity matrix, then remove rows for non-zero weight vectors.
gate_x1_zero_weight_vectors = [
    # a_ij short edges besides a11 and a12
    short_edge(i,j)+[0]*(18+24) for i in [1,2,3,4] for j in [1,2,3] if (i,j)!= (1,1) and (i,j) != (1,2)
]+[ # A_k long edges besides A5
    long_edge(k)+[0]*(18+24) for k in range(1,6+1) if k != 5
]+[ # b_ij short edges
    [0]*18 + short_edge(i,j) + [0]*24 for i in [1,2,3,4] for j in [1,2,3]
]+[ #B_k long edges
    [0]*18 + long_edge(k) + [0]*24 for k in range(1,6+1)
]+[ # threads besides x1 and z2
    [0]*(18+18) + list(std_basis(i,24)) for i in range(24) if std_basis(i,24) != x1vec and std_basis(i,24) != z2vec
]
    
gate_x1_zero_weight_space = get_centralizer(gate_x1_zero_weight_vectors,omega_41)

In [ ]:
print(gate_x1_zero_weight_space.basis_matrix().LLL())

In [ ]:
# the non-zero weights:
print(coeffs*omega_41*std_basis(0,60),gate_x1_weight*std_basis(0,60))
print(coeffs*omega_41*std_basis(1,60),gate_x1_weight*std_basis(1,60))
print(coeffs*omega_41*vector([0]*(18+18)+list(x1vec)),gate_x1_weight*vector([0]*(18+18)+list(x1vec)))
print(coeffs*omega_41*vector([0]*(18+18)+list(z2vec)),gate_x1_weight*vector([0]*(18+18)+list(z2vec)))
f = (coeffs*omega_41*std_basis(0,60))[0]
g = (gate_x1_weight*std_basis(0,60))[0]

# turn them into a system of equations: This is having trouble, since sage doesn't recognize these as symbolic expressions.
#solve([coeffs*omega_41*std_basis(0,60)[0]==gate_x1_weight*std_basis(0,60)[0]],a12,a13,A4,A5,x1,x2,y1,w1)
print(coeffs*omega_41*std_basis(0,60)[0]==gate_x1_weight*std_basis(0,60)[0])
solve([-a12 + a13 - A4 - A5 - x1 + x2 - y1 - z1 - w1==-1,y1==2],y1,x1)

On Ubuntu 22.04 and SageMath 9.5 I was able to install SnapPy with the terminal command:
```
sage -pip install --upgrade --user snappy
```

In [ ]:
import snappy
M = snappy.Triangulation('4_1')
gluing = M._get_tetrahedra_gluing_data()
print(gluing)
print(M._to_string())

In [ ]:
for i in range(4):
    print(i, gluing[0][1][i])
    for j in range(4):
        if i != j:
            print("a{0}{1}".format(i,j),"glues to", "b{0}{1}".format(gluing[0][1][i][i],gluing[0][1][i][j]))

In [ ]:
print(M._get_tetrahedra_gluing_data())

In [ ]:
snappy.Manifold(M).browse()
%gui tk

In [ ]:
print(M._to_string())
M.reverse_orientation()
print(M._to_string())

In [ ]:
import snappy_manifolds
Mc = snappy.snap.t3mlite.Mcomplex(M)
print("Gluing info:")
for tet in Mc.Tetrahedra:
    print(tet.Gluing)

t1 = Mc.Tetrahedra[0]

e1 = Mc.Edges[0]
print("Embeddings:")
for em in e1.embeddings():
    print(em)

print(e1.info())
a = e1.get_arrow() # I think arrow tells me <Edge | Face | Tet >

In [ ]:
print(M._to_string())

In [ ]:
import snappy
snappy.snap.t3mlite.Mcomplex?

# q = 1 Version  - NOT UPDATED

Here's the plan:
1. Work with a (Laurent?) polynomial ring in all the edge variables.
2. Quotient by the ideal generated by all the relations. By "all the relations" I mean $m_i - 1$ for all monomial relations $m_i$, plus the great circles which are already polynomials.


There's a silly issue - both R and Rnew have the variables x1,...,z6, and so sometimes think you're trying to multiple elements in different rings. 

TODO - I expect the quotient to need three generators, but sagemath just uses the images of the generators. Figure out how to get this to work. Playing around with ideals and reduce makes me think I might need to work with polynomial rings instead of laurent ones.

## With Polynomials

In [ ]:
# Some recap.
Rnew # This is the ring with all generators for both tetrahedra and also the threads.
Rpoly = R.polynomial_ring()

# all the monomial relations, I need to convert them into elements in the ring.
list_of_gluing_constraints_41;
list_of_monodromy_constraints_41;
list_of_thread_monodromy_constraints_41;


In [ ]:
# scratch
test_constraint = "*".join(["{0}^{1}".format(var,power) for var,power in list(zip(lex_order_41[1:],list_of_gluing_constraints_41[1][1:])) if power != 0])
#lattice_coord_to_free_algebra_element(list_of_gluing_constraints_41[2][1:],lex_order_41[1:],R)

R(a43^-1*R(x1))

In [ ]:
monomial_constraints = [
    lattice_coord_to_free_algebra_element(
        constraint[1:],
        lex_order_41[1:],
        R
    ) -1 for constraint in list_of_gluing_constraints_41 + list_of_monodromy_constraints_41 + list_of_thread_monodromy_constraints_41]

monomial_constraint_ideal = R.ideal(monomial_constraints)
glued_algebra = R.quotient_ring(monomial_constraint_ideal)

In [ ]:
# A function to clear denominators.

def clear_denominator(laurent_poly,ring):
    """multiplies laurent_poly by a monomial to clear all negative powers.
    
    Parameters:
    laurent_poly (laurent_polynomial) - needs clearing
    ring (LaurentPolynomialRing) - the ambient ring
    
    Returns:
    polynomial version of laurent_poly
    """
    powers_grouped_by_variable = list(zip(*laurent_poly.exponents()))    
    lowest_exponents = [reduce(lambda x,y : -min(x,y), l) for l in powers_grouped_by_variable]
    poly_ring = ring.polynomial_ring()
    
    return poly_ring(ring.monomial(*lowest_exponents)*laurent_poly)

    
clear_denominator(R(a43^-1*R(x1) -a21^-2*a43^4),R)

In [ ]:
# now lets look at the quotient in a polynomial ring.
positive_powers_constraints = [clear_denominator(f,R) for f in monomial_constraints]

In [ ]:
poly_constraints_ideal = Rpoly.ideal(positive_powers_constraints)
poly_glued_algebra = Rpoly.quotient(poly_constraints_ideal)